In [ ]:
# ================================
# 📌 Cell 1: Mount Drive + Imports
# ================================
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
import cv2
import random
import numpy as np
from tqdm import tqdm

print("Drive mounted and libraries imported.")

Mounted at /content/drive
Drive mounted and libraries imported.


In [ ]:
# ====================================================
# 📌 Cell 2: Define Source Dataset & Output Directories
# ====================================================

# Correct path to the REAL BDD100K folder (NOT shortcut)
SRC_IMG_DIR = "/content/drive/MyDrive/bdd100k-yolo/train/images"

# Output Pix2Pix dataset root
P2P_ROOT = "/content/pix2pix_bdd"
TRAIN_DIR = os.path.join(P2P_ROOT, "train")

os.makedirs(TRAIN_DIR, exist_ok=True)

print("Source images:", SRC_IMG_DIR)
print("Pix2Pix dataset will be saved at:", TRAIN_DIR)


Source images: /content/drive/MyDrive/bdd100k-yolo/train/images
Pix2Pix dataset will be saved at: /content/pix2pix_bdd/train


In [ ]:
# ===============================================
# 📌 Cell 3: Extreme Realistic Weather Transforms
# ===============================================

import numpy as np
import cv2
import random


# ---------------------------------------------------
# 🌫 1. EXTREME FOG (Milky + distance haze)
# ---------------------------------------------------
def fog_effect(img, intensity=0.85):
    h, w = img.shape[:2]

    fog = np.random.normal(235, 12, img.shape).astype(np.uint8)
    fog = cv2.GaussianBlur(fog, (0,0), 120)

    blended = cv2.addWeighted(img, 1-intensity, fog, intensity, 0)

    gradient = np.tile(np.linspace(1.8, 0.4, h).reshape(h,1), (1,w))
    gradient = np.dstack([gradient]*3)

    out = (blended * gradient).clip(0,255).astype(np.uint8)
    return out


# ---------------------------------------------------
# 🌙 2. EXTREME LOW-LIGHT (gamma + noise)
# ---------------------------------------------------
def low_light(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsv[:,:,2] = (hsv[:,:,2] * 0.22).clip(0,255)
    hsv[:,:,1] = (hsv[:,:,1] * 0.45).clip(0,255)
    dark = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    noise = np.random.normal(0, 18, img.shape).astype(np.int16)
    return np.clip(dark + noise, 0, 255).astype(np.uint8)

# ===============================================
# 📌 Cell 3: Extreme Realistic Weather Transforms
# ===============================================

import numpy as np
import cv2
import random


# ---------------------------------------------------
# 🌫 1. EXTREME FOG (Milky + distance haze)
# ---------------------------------------------------
def fog_effect(img, intensity=0.85):
    h, w = img.shape[:2]

    fog = np.random.normal(235, 12, img.shape).astype(np.uint8)
    fog = cv2.GaussianBlur(fog, (0,0), 120)

    blended = cv2.addWeighted(img, 1-intensity, fog, intensity, 0)

    gradient = np.tile(np.linspace(1.8, 0.4, h).reshape(h,1), (1,w))
    gradient = np.dstack([gradient]*3)

    out = (blended * gradient).clip(0,255).astype(np.uint8)
    return out


# ---------------------------------------------------
# 🌙 2. EXTREME LOW-LIGHT (gamma + noise)
# ---------------------------------------------------
def low_light(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsv[:,:,2] = (hsv[:,:,2] * 0.22).clip(0,255)
    hsv[:,:,1] = (hsv[:,:,1] * 0.45).clip(0,255)
    dark = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    noise = np.random.normal(0, 18, img.shape).astype(np.int16)
    return np.clip(dark + noise, 0, 255).astype(np.uint8)


# ---------------------------------------------------
# 🌞 3. NEW REALISTIC SUN GLARE (direction + bloom)
# ---------------------------------------------------
def sun_glare(img):
    h, w = img.shape[:2]
    overlay = img.copy().astype(np.float32)

    # Sun top-right or top-left
    cx = random.choice([random.randint(0, int(w*0.3)),
                        random.randint(int(w*0.7), w-1)])
    cy = random.randint(0, int(h*0.25))

    # Large bloom
    radius = random.randint(180, 300)
    cv2.circle(overlay, (cx, cy), radius, (255,255,255), -1)

    # Horizontal glare streak
    cv2.line(overlay, (0, cy), (w, cy), (255,240,200), 25)

    # Vertical glare streak
    cv2.line(overlay, (cx, 0), (cx, h), (255,240,200), 18)

    # Soft glow halo
    halo = cv2.GaussianBlur(overlay, (0, 0), sigmaX=50)
    out = cv2.addWeighted(img.astype(np.float32), 0.45, halo, 0.55, 0)

    return out.astype(np.uint8)


# ---------------------------------------------------
# 🚗 4. EXTREME MOTION BLUR (horizontal shake)
# ---------------------------------------------------
def motion_blur(img):
    k = random.choice([25,35,45])
    kernel = np.zeros((k, k))
    kernel[int(k/2), :] = 1
    kernel /= k
    blur = cv2.filter2D(img, -1, kernel)

    M = np.float32([[1,0,random.randint(-3,3)], [0,1,random.randint(-2,2)]])
    return cv2.warpAffine(blur, M, (img.shape[1], img.shape[0]))


# ---------------------------------------------------
# 🌧 5. EXTREME HEAVY RAIN (streaks + motion)
# ---------------------------------------------------
def rain_streaks(img):
    h, w = img.shape[:2]
    rain = img.copy()

    for drops, length, angle in [
        (2600, 18, -20),
        (2000, 25, -30),
        (1500, 30, -10)
    ]:
        for _ in range(drops):
            x = random.randint(0,w)
            y = random.randint(0,h)
            ex = int(x + length*np.cos(np.radians(angle)))
            ey = int(y + length*np.sin(np.radians(angle)))
            cv2.line(rain, (x,y), (ex,ey), (220,220,220), 2)

    k = 18
    kernel = np.zeros((k,k))
    kernel[:,int(k/2)] = 1
    kernel /= k
    return cv2.filter2D(rain, -1, kernel)


# ---------------------------------------------------
# ❄️ 6. FROST (Cracked ice pattern)
# ---------------------------------------------------
def frost_effect(img):
    h, w = img.shape[:2]
    frost = cv2.imread("/content/drive/MyDrive/frost_texture.jpg")

    if frost is None:
        frost = np.random.normal(240, 10, img.shape).astype(np.uint8)
        frost = cv2.GaussianBlur(frost, (0,0), 70)

    frost = cv2.resize(frost, (w,h))
    cracks = frost.copy()

    for _ in range(120):
        x1, y1 = random.randint(0,w), random.randint(0,h)
        x2, y2 = x1+random.randint(-120,120), y1+random.randint(-120,120)
        cv2.line(cracks, (x1,y1), (x2,y2), (255,255,255), 1)

    return cv2.addWeighted(img, 0.55, cracks, 0.65, 0)


# ---------------------------------------------------
# ❄️ 7. SNOWFALL (depth layering)
# ---------------------------------------------------
def snow_effect(img):
    h, w = img.shape[:2]
    snow = img.copy()

    # foreground flakes
    for _ in range(900):
        cv2.circle(snow, (random.randint(0,w), random.randint(0,h)),
                   random.randint(2,5), (255,255,255), -1)

    # background flakes
    for _ in range(3500):
        cv2.circle(snow, (random.randint(0,w), random.randint(0,h)),
                   random.randint(1,2), (230,230,230), -1)

    # falling motion blur
    k = 20
    kernel = np.zeros((k,k))
    kernel[:,int(k/2)] = 1
    kernel /= k
    snow = cv2.filter2D(snow, -1, kernel)

    return cv2.addWeighted(img, 0.7, snow, 0.5, 0)


# ---------------------------------------------------
# 🎛 RANDOM TRANSFORM PICKER
# ---------------------------------------------------
def random_transform(img):
    transform = random.choice([
        fog_effect,
        low_light,
        sun_glare,
        motion_blur,
        rain_streaks,
        frost_effect,
        snow_effect,
    ])
    return transform(img)


# ---------------------------------------------------
# 🚗 4. EXTREME MOTION BLUR (horizontal shake)
# ---------------------------------------------------
def motion_blur(img):
    k = random.choice([25,35,45])
    kernel = np.zeros((k, k))
    kernel[int(k/2), :] = 1
    kernel /= k
    blur = cv2.filter2D(img, -1, kernel)

    M = np.float32([[1,0,random.randint(-3,3)], [0,1,random.randint(-2,2)]])
    return cv2.warpAffine(blur, M, (img.shape[1], img.shape[0]))


# ---------------------------------------------------
# 🌧 5. EXTREME HEAVY RAIN (streaks + motion)
# ---------------------------------------------------
def rain_streaks(img):
    h, w = img.shape[:2]
    rain = img.copy()

    for drops, length, angle in [
        (2600, 18, -20),
        (2000, 25, -30),
        (1500, 30, -10)
    ]:
        for _ in range(drops):
            x = random.randint(0,w)
            y = random.randint(0,h)
            ex = int(x + length*np.cos(np.radians(angle)))
            ey = int(y + length*np.sin(np.radians(angle)))
            cv2.line(rain, (x,y), (ex,ey), (220,220,220), 2)

    k = 18
    kernel = np.zeros((k,k))
    kernel[:,int(k/2)] = 1
    kernel /= k
    return cv2.filter2D(rain, -1, kernel)


# ---------------------------------------------------
# ❄️ 6. FROST (Cracked ice pattern)
# ---------------------------------------------------
def frost_effect(img):
    h, w = img.shape[:2]
    frost = cv2.imread("/content/drive/MyDrive/frost_texture.jpg")

    if frost is None:
        frost = np.random.normal(240, 10, img.shape).astype(np.uint8)
        frost = cv2.GaussianBlur(frost, (0,0), 70)

    frost = cv2.resize(frost, (w,h))
    cracks = frost.copy()

    for _ in range(120):
        x1, y1 = random.randint(0,w), random.randint(0,h)
        x2, y2 = x1+random.randint(-120,120), y1+random.randint(-120,120)
        cv2.line(cracks, (x1,y1), (x2,y2), (255,255,255), 1)

    return cv2.addWeighted(img, 0.55, cracks, 0.65, 0)


# ---------------------------------------------------
# ❄️ 7. SNOWFALL (depth layering)
# ---------------------------------------------------
def snow_effect(img):
    h, w = img.shape[:2]
    snow = img.copy()

    # foreground flakes
    for _ in range(900):
        cv2.circle(snow, (random.randint(0,w), random.randint(0,h)),
                   random.randint(2,5), (255,255,255), -1)

    # background flakes
    for _ in range(3500):
        cv2.circle(snow, (random.randint(0,w), random.randint(0,h)),
                   random.randint(1,2), (230,230,230), -1)

    # falling motion blur
    k = 20
    kernel = np.zeros((k,k))
    kernel[:,int(k/2)] = 1
    kernel /= k
    snow = cv2.filter2D(snow, -1, kernel)

    return cv2.addWeighted(img, 0.7, snow, 0.5, 0)


# ---------------------------------------------------
# 🎛 RANDOM TRANSFORM PICKER
# ---------------------------------------------------
def random_transform(img):
    transform = random.choice([
        fog_effect,
        low_light,
        sun_glare,
        motion_blur,
        rain_streaks,
        frost_effect,
        snow_effect,
    ])
    return transform(img)


In [ ]:
# =======================================
# 📌 Cell 4: Select 1000 random images
# =======================================

all_images = sorted(glob.glob(os.path.join(SRC_IMG_DIR, "*.jpg")))

print("Total images found:", len(all_images))

random.seed(42)
selected_images = random.sample(all_images, 1000)

print("Selected 1000 images successfully.")

Total images found: 70000
Selected 1000 images successfully.


In [ ]:
# ============================================
# 📌 Create Pix2Pix AB pairs + YOLO B images
# ============================================

import os
import glob
import shutil
from tqdm import tqdm
import cv2
import numpy as np

# --- paths (edit if needed) ---
SRC_IMG_DIR = "/content/drive/MyDrive/bdd100k-yolo/train/images"
SRC_LBL_DIR = "/content/drive/MyDrive/bdd100k-yolo/train/labels"

P2P_ROOT   = "/content/pix2pix_bdd"              # for Pix2Pix AB images
P2P_TRAIN  = os.path.join(P2P_ROOT, "train")     # AB images here

YOLO_IMG_OUT = "/content/pix2pix_bdd/yolo_images"  # YOLO-ready B images
YOLO_LBL_OUT = "/content/pix2pix_bdd/yolo_labels"  # copied labels

os.makedirs(P2P_TRAIN, exist_ok=True)
os.makedirs(YOLO_IMG_OUT, exist_ok=True)
os.makedirs(YOLO_LBL_OUT, exist_ok=True)

print("Pix2Pix AB dir:", P2P_TRAIN)
print("YOLO B images dir:", YOLO_IMG_OUT)
print("YOLO labels dir:", YOLO_LBL_OUT)

# --------------------------------------
# 1️⃣ Collect all images with labels
# --------------------------------------
all_imgs = sorted(glob.glob(os.path.join(SRC_IMG_DIR, "*.jpg")))
paired_imgs = []

for img_path in all_imgs:
    base = os.path.splitext(os.path.basename(img_path))[0]
    lbl_path = os.path.join(SRC_LBL_DIR, base + ".txt")
    if os.path.exists(lbl_path):
        paired_imgs.append((img_path, lbl_path))

print(f"Total images with labels found: {len(paired_imgs)}")

# Optional: limit to first 1000 images
MAX_SAMPLES = 1000
if len(paired_imgs) > MAX_SAMPLES:
    paired_imgs = random.sample(paired_imgs, MAX_SAMPLES)
    print(f"Using a random subset of {MAX_SAMPLES} images with labels.")


# --------------------------------------
# 2️⃣ Define mapping modes (Clear → X)
# --------------------------------------
# Each entry: name : transform_function
p2p_modes = {
    "fog":       fog_effect,     # Clear → Fog
    "rain":      rain_streaks,   # Clear → Rain
    "snow":      snow_effect,    # Clear → Snow
    "lowlight":  low_light,      # Clear → Low-light (Night-ish)
    "glare":     sun_glare,      # Clear → Glare
    # You can treat "day→night" as stronger low_light
    "night":     low_light,      # alias
    # "summer" / "winter" can be approximated via color shifts if you want
}


def apply_style(img, mode):
    """Apply the right transform based on mode name."""
    if mode == "night":
        # extra dark variant of low_light
        out = low_light(img)
        out = low_light(out)  # darken twice
        return out
    # For now, summer/winter not modeled separately – could be added.
    return p2p_modes[mode](img)


# --------------------------------------
# 3️⃣ Create Pix2Pix AB images + YOLO B images
# --------------------------------------
def create_pix2pix_and_yolo(img_path, lbl_path):
    base = os.path.splitext(os.path.basename(img_path))[0]

    # load original (A) in original size
    A_full = cv2.imread(img_path)
    if A_full is None:
        return

    A_full_rgb = cv2.cvtColor(A_full, cv2.COLOR_BGR2RGB)

    # A resized for Pix2Pix (256x256)
    A_res = cv2.resize(A_full_rgb, (256, 256))

    for mode_name in p2p_modes.keys():
        # --- create B image in original size for YOLO ---
        B_full_rgb = apply_style(A_full_rgb, mode_name)
        B_full_bgr = cv2.cvtColor(B_full_rgb, cv2.COLOR_RGB2BGR)

        # save YOLO B image
        yolo_img_name = f"{base}_{mode_name}_p2p.jpg"
        yolo_img_path = os.path.join(YOLO_IMG_OUT, yolo_img_name)
        cv2.imwrite(yolo_img_path, B_full_bgr)

        # copy label with same name
        yolo_lbl_name = f"{base}_{mode_name}_p2p.txt"
        yolo_lbl_path = os.path.join(YOLO_LBL_OUT, yolo_lbl_name)
        shutil.copy(lbl_path, yolo_lbl_path)

        # --- create Pix2Pix AB image at 256x256 ---
        B_res = cv2.resize(B_full_rgb, (256, 256))
        AB = np.concatenate([A_res, B_res], axis=1)   # [A | B]
        AB_bgr = cv2.cvtColor(AB, cv2.COLOR_RGB2BGR)

        p2p_name = f"{base}_{mode_name}.jpg"
        p2p_path = os.path.join(P2P_TRAIN, p2p_name)
        cv2.imwrite(p2p_path, AB_bgr)


# run over all paired images
for img_path, lbl_path in tqdm(paired_imgs, desc="Building Pix2Pix + YOLO datasets"):
    create_pix2pix_and_yolo(img_path, lbl_path)

print("✅ Done: Pix2Pix AB pairs + YOLO-ready B images created!")


Pix2Pix AB dir: /content/pix2pix_bdd/train
YOLO B images dir: /content/pix2pix_bdd/yolo_images
YOLO labels dir: /content/pix2pix_bdd/yolo_labels
Total images with labels found: 70000
Using a random subset of 1000 images with labels.


Building Pix2Pix + YOLO datasets: 100%|██████████| 1000/1000 [2:00:27<00:00,  7.23s/it]

✅ Done: Pix2Pix AB pairs + YOLO-ready B images created!


In [ ]:
!zip -r pix2pix_bdd.zip pix2pix_bdd/

Streaming output truncated to the last 5000 lines.
  adding: pix2pix_bdd/train/2aecd572-9a5d712c_rain.jpg (deflated 0%)
  adding: pix2pix_bdd/train/09b978e2-d23f214b_lowlight.jpg (deflated 0%)
  adding: pix2pix_bdd/train/ad33b098-18ff10e5_rain.jpg (deflated 0%)
  adding: pix2pix_bdd/train/fe16aa77-810c2e1f_lowlight.jpg (deflated 0%)
  adding: pix2pix_bdd/train/672363b5-9b0e4dae_glare.jpg (deflated 0%)
  adding: pix2pix_bdd/train/022ff54c-3cb58ec1_fog.jpg (deflated 2%)
  adding: pix2pix_bdd/train/36f91f1f-9549b3ad_rain.jpg (deflated 0%)
  adding: pix2pix_bdd/train/581101cc-8ef51e1d_glare.jpg (deflated 1%)
  adding: pix2pix_bdd/train/12dc1ad6-e8d7e789_lowlight.jpg (deflated 0%)
  adding: pix2pix_bdd/train/62a5ec72-faf4fd7b_fog.jpg (deflated 1%)
  adding: pix2pix_bdd/train/af97a961-1770be3c_glare.jpg (deflated 1%)
  adding: pix2pix_bdd/train/5ea96a8f-e0616451_night.jpg (deflated 0%)
  adding: pix2pix_bdd/train/64dfa7fa-d85abebf_rain.jpg (deflated 0%)
  adding: pix2pix_bdd/train/0e363dc5-c

In [ ]:
from google.colab import files
files.download('pix2pix_bdd.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>